In [1]:
import pandas as pd
import ta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from pybit import spot  # <-- import HTTP & WSS for spot
from pybit import HTTP  # supports inverse perp & futures, usdt perp, spot.
from dotenv import load_dotenv
import os
import warnings
warnings.simplefilter("ignore")

In [2]:
load_dotenv()

True

In [3]:
api_key_pw = os.getenv('api_key_bot_IP')
api_secret_pw = os.getenv('api_secret_bot_IP')

In [4]:
from pybit import spot
session_auth = spot.HTTP(
    endpoint='https://api.bybit.com',
    api_key = api_key_pw,
    api_secret= api_secret_pw
)

In [5]:
def get5minutedata(symbol, interval):
    frame = pd.DataFrame(session_auth.query_kline(symbol=symbol, interval=interval)["result"])
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index("Time")
    frame.index = pd.to_datetime(frame.index, unit="ms")
    frame = frame.astype(float)
    return frame

In [6]:
df = get5minutedata("SOLUSDT", "5m")
df

,Open,High,Low,Close,Volume
Time,,,,,
2023-01-14 11:35:00,22.25,22.27,22.11,22.14,3166.158
2023-01-14 11:40:00,22.14,22.21,22.04,22.14,3240.960
2023-01-14 11:45:00,22.14,22.24,22.13,22.13,4264.050
2023-01-14 11:50:00,22.13,22.17,22.11,22.14,5756.265
2023-01-14 11:55:00,22.14,22.42,22.14,22.39,8611.840
...,...,...,...,...,...
2023-01-17 22:30:00,23.23,23.27,23.21,23.21,718.879
2023-01-17 22:35:00,23.21,23.25,23.15,23.24,1449.883
2023-01-17 22:40:00,23.24,23.24,23.22,23.23,115.373


In [7]:
sol = pd.read_csv ('./sol_todo.csv')
sol['Time'] = sol['time'].str.extract('(\d{4}-\d{2}-\d{2}\w\d{2}:\d{2}:\d{2})', expand=True)
sol['Time'] = pd.to_datetime(sol["Time"])
sol.set_index("Time", inplace = True)
sol.drop(columns="time", inplace=True)
sol.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
sol = sol[7300:]

In [8]:
sol

,Open,High,Low,Close,Volume
Time,,,,,
2022-12-30 09:20:00,9.070,9.090,9.035,9.055,199329.7
2022-12-30 09:25:00,9.055,9.125,9.050,9.090,249521.9
2022-12-30 09:30:00,9.090,9.110,9.040,9.080,196293.6
2022-12-30 09:35:00,9.080,9.085,8.965,8.980,378871.2
2022-12-30 09:40:00,8.980,9.045,8.975,9.045,246094.2
...,...,...,...,...,...
2023-01-10 15:20:00,16.115,16.145,16.045,16.140,169186.0
2023-01-10 15:25:00,16.140,16.145,16.080,16.120,64190.9
2023-01-10 15:30:00,16.120,16.315,16.115,16.285,342420.7


In [9]:
class Sol_long(Strategy):

    def init(self):
        close = self.data.Close
        self.rsi= self.I(ta.momentum.rsi, pd.Series(close), window=14)
        self.K = self.I(ta.momentum.stochrsi_k, pd.Series(close), window= 14)
        self.D = self.I(ta.momentum.stochrsi_d, pd.Series(close), window= 14)

    def next(self):
        price = self.data.Close
        rsi = self.rsi
        K = self.K
        D = self.D
       
        if  rsi < 32 and K > D and K < 0.15:
            limit_price = price * 0.995
            sl = limit_price * 0.98
            tp = limit_price * 1.05
            self.buy(sl=sl, tp=tp)
    

In [10]:
bt = Backtest(sol, Sol_long, cash= 10000, commission= 0.0015)

In [11]:
output = bt.run()

In [12]:
output

Start                     2022-12-30 09:20:00
End                       2023-01-10 15:40:00
Duration                     11 days 06:20:00
Exposure Time [%]                   33.990755
Equity Final [$]                 12387.280754
Equity Peak [$]                  12452.135754
Return [%]                          23.872808
Buy & Hold Return [%]               78.741027
Return (Ann.) [%]                67200.674655
Volatility (Ann.) [%]            23018.712539
Sharpe Ratio                         2.919393
Sortino Ratio                    41794.271841
Calmar Ratio                     18406.884318
Max. Drawdown [%]                   -3.650845
Avg. Drawdown [%]                   -0.687363
Max. Drawdown Duration        1 days 07:15:00
Avg. Drawdown Duration        0 days 01:47:00
# Trades                                    7
Win Rate [%]                        85.714286
Best Trade [%]                       4.318522
Worst Trade [%]                     -2.636046
Avg. Trade [%]                    

In [13]:
bt.plot()

Row(id='1611', ...)